# Defiine params and lib

In [17]:
import requests
from io import BytesIO

from PIL import Image
import numpy as np

In [18]:
# Parameters
input_size = (160,160)

#define input shape
channel = (3,)
input_shape = input_size + channel

#define labels
labels = ['Keriting', 'Kuning', 'Layu']

# Define preprocess function

In [19]:
def preprocess(img,input_size):
    nimg = img.convert('RGB').resize(input_size, resample= 0)
    img_arr = (np.array(nimg))/255
    return img_arr

def reshape(imgs_arr):
    return np.stack(imgs_arr, axis=0)

# Load Model Langsung Kesini aja

In [26]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

# Define the swish activation function
def swish(x):
    return x * tf.keras.backend.sigmoid(x)

# Define or import your FixedDropout layer here if needed
class FixedDropout(tf.keras.layers.Dropout):
    # Custom implementation or just inherit Dropout
    pass

# Define the input size based on the model's expected input dimensions
input_size = (160, 160)  # Sesuaikan dengan input size yang diharapkan model Anda

# Load the model
MODEL_PATH = 'model/Cabe/model_cabe.h5'
# MODEL_PATH = 'best_model.keras'

try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish, 'FixedDropout': FixedDropout})
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)

# Define preprocess function
def preprocess(image, input_size):
    # Resize image to match input size
    image = image.resize(input_size)
    # Convert image to numpy array and normalize to [0, 1]
    image = np.array(image).astype(np.float32) / 255.0
    # Add batch dimension for model prediction
    return np.expand_dims(image, axis=0)

# Load and preprocess the image
im = Image.open('k2.jpg')
X = preprocess(im, input_size)  # Preprocess the image

# Predict with the model
y = model.predict(X)

# Assuming labels list is defined (e.g., labels = ["label_1", "label_2", "label_3"])
labels = ['Keriting', 'Kuning', 'Layu']  # Sesuaikan dengan label model Anda

# Print prediction result
print("Predicted label:", labels[np.argmax(y)], "Confidence:", np.max(y))


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted label: Keriting Confidence: 0.99999285


USing tflite

In [6]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Define the input size based on the model's expected input dimensions
input_size = (160, 160)  # Sesuaikan dengan input size yang diharapkan model Anda

# Define preprocess function
def preprocess(image, input_size):
    # Resize image to match input size
    image = image.resize(input_size)
    # Convert image to numpy array and normalize to [0, 1]
    image = np.array(image).astype(np.float32) / 255.0
    # Add batch dimension for model prediction
    return np.expand_dims(image, axis=0)

# Load model .tflite with TensorFlow Lite Interpreter
MODEL_PATH = 'model/Rupiah_Project/model_uang.tflite'
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
im = Image.open('5 2.jpg')
X = preprocess(im, input_size)  # Preprocess the image

# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], X)

# Run inference
interpreter.invoke()

# Retrieve the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])
predicted_label = np.argmax(output_data)
confidence_score = np.max(output_data)

# Assuming labels list is defined
labels = ['100K', '10k', '1k', '20K', '2k', '50K', '5k']  # Sesuaikan dengan label model Anda

# Print prediction result
print("Predicted label:", labels[predicted_label], "Confidence:", confidence_score)


Predicted label: 5k Confidence: 1.0


In [6]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from PIL import Image

# Define the swish activation function
def swish(x):
    return x * tf.keras.backend.sigmoid(x)

# Define or import your FixedDropout layer here if needed
class FixedDropout(tf.keras.layers.Dropout):
    pass

# Define the input size based on the model's expected input dimensions
input_size = (160, 160)

# Load the model
MODEL_PATH = 'model/Rupiah_Project/model_uang.h5'
try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish, 'FixedDropout': FixedDropout})
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)

# Define preprocess function
def preprocess(image, input_size):
    image = image.resize(input_size)
    image = np.array(image).astype(np.float32) / 255.0
    return np.expand_dims(image, axis=0)

# Define labels for predictions
labels = ['100K', '10k', '1k', '20K', '2k', '50K', '5k']

# Start live video capture
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Convert the frame to RGB format for PIL processing
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img)

        # Preprocess the image for model prediction
        X = preprocess(pil_img, input_size)
        y = model.predict(X)

        # Get the predicted label and confidence
        predicted_label = labels[np.argmax(y)]
        confidence = np.max(y)

        # Display the prediction on the video frame
        cv2.putText(frame, f"Label: {predicted_label}, Confidence: {confidence:.2f}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show the video with predictions
        cv2.imshow('Live Money Detection', frame)

        # Press 'q' to quit the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

# Using Live Cam

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from PIL import Image

# Define the swish activation function
def swish(x):
    return x * tf.keras.backend.sigmoid(x)

# Define or import your FixedDropout layer here if needed
class FixedDropout(tf.keras.layers.Dropout):
    pass

# Define the input size based on the model's expected input dimensions
input_size = (160, 160)

# Load the model
MODEL_PATH = 'model/Rupiah_Project/model_uang.h5'
try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish, 'FixedDropout': FixedDropout})
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)

# Define preprocess function
def preprocess(image, input_size):
    image = image.resize(input_size)
    image = np.array(image).astype(np.float32) / 255.0
    return np.expand_dims(image, axis=0)

# Define labels for predictions
labels = ['100K', '10k', '1k', '20K', '2k', '50K', '5k']

# Start live video capture
cap = cv2.VideoCapture(1)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # Convert the frame to RGB format for PIL processing
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(img)

        # Preprocess the image for model prediction
        X = preprocess(pil_img, input_size)
        y = model.predict(X)

        # Get the predicted label and confidence
        predicted_label = labels[np.argmax(y)]
        confidence = np.max(y)

        # Display the prediction on the video frame
        cv2.putText(frame, f"Label: {predicted_label}, Confidence: {confidence:.2f}",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show the video with predictions
        cv2.imshow('Live Money Detection', frame)

        # Press 'q' to quit the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

# Mencoba Menggunakan Kamera HP

In [10]:
import cv2
import numpy as np

# Ganti URL ini dengan URL dari kamera HP Anda, jika menggunakan IP Webcam atau DroidCam
cap = cv2.VideoCapture('http://172.27.80.191:8080')

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Ubah frame ke ruang warna HSV (untuk deteksi warna yang lebih mudah)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Definisikan rentang warna yang sesuai dengan warna dominan uang (misalnya biru/hijau)
    lower_color = np.array([30, 40, 40])  # Ganti dengan rentang warna yang sesuai
    upper_color = np.array([90, 255, 255])
    
    # Mask untuk warna uang
    mask = cv2.inRange(hsv, lower_color, upper_color)
    
    # Lakukan deteksi tepi dengan Canny
    edges = cv2.Canny(mask, 100, 200)
    
    # Cari kontur di gambar tepi
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        # Approximate contour ke bentuk persegi empat (untuk uang kertas)
        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
        
        # Cek apakah bentuknya persegi panjang dan ukurannya cukup besar
        if len(approx) == 4 and cv2.contourArea(cnt) > 1000:
            # Gambar bounding box di sekitar kontur
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # Potong area uang untuk prediksi lebih lanjut
            roi = frame[y:y+h, x:x+w]
            
            # Preprocessing ROI dan lakukan prediksi model (sesuaikan dengan model Anda)
            # Example: X = preprocess(roi, input_size); y = model.predict(X)
            # Tampilkan label prediksi pada bounding box (contoh "Uang Terdeteksi")
            cv2.putText(frame, "Uang Terdeteksi", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Tampilkan hasil frame
    cv2.imshow("Live Money Detection with Boundary", frame)

    # Break the loop with 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


# Load models

In [4]:
from tensorflow.keras.models import load_model

In [5]:
from keras import backend as K

def swish(x):
    return x * K.sigmoid(x)


In [6]:
from tensorflow.keras.models import load_model

MODEL_PATH = 'model/Rupiah_Project/model_uang.tflite'
try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish})
    print("Model loaded successfully.")
    # Proceed with predictions here if loading succeeds
except Exception as e:
    print("Error loading the model:", e)
    # Optionally, you can attempt an alternative loading or handle the error


Error loading the model: File format not supported: filepath=model/Rupiah_Project/model_uang.tflite. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(model/Rupiah_Project/model_uang.tflite, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).


In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout  # Modify if you have a specific implementation for FixedDropout

# Define or import the custom swish activation function and FixedDropout layer here if needed
def swish(x):
    return x * tf.keras.backend.sigmoid(x)

class FixedDropout(Dropout):
    # Assuming FixedDropout extends Dropout and has custom behavior, define it here
    pass

MODEL_PATH = 'model/medium_project/model_medium.h5'
try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish, 'FixedDropout': FixedDropout})
    print("Model loaded successfully.")
    # Proceed with predictions here if loading succeeds
except Exception as e:
    print("Error loading the model:", e)


Model loaded successfully.


# Predict the image

In [7]:
# read image
im = Image.open('contoh_prediksi.jpg')
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

tulip 0.92026424


In [8]:
y

array([[7.9548061e-02, 1.8205217e-04, 3.7672000e-06, 1.9034776e-06,
        9.2026424e-01]], dtype=float32)

In [9]:
print( labels[np.argmax(y)], np.max(y) )

tulip 0.92026424


In [18]:
# read image
im = Image.open('dataset/train/1k/1000_2_jpg.rf.02b4f767483f179bd40d1c31624c24a1.jpg')
X = preprocess(im,input_size)
X = reshape([X])
y = model.predict(X)

print( labels[np.argmax(y)], np.max(y) )

NameError: Exception encountered when calling Activation.call().

[1mname 'tf' is not defined[0m

Arguments received by Activation.call():
  • inputs=tf.Tensor(shape=(1, 80, 80, 32), dtype=float32)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

# Define the swish activation function
def swish(x):
    return x * tf.keras.backend.sigmoid(x)

# Define or import your FixedDropout layer here if needed
class FixedDropout(tf.keras.layers.Dropout):
    # Custom implementation or just inherit Dropout
    pass

# Load the model
MODEL_PATH = 'model/medium_project/model_medium.h5'
try:
    model = load_model(MODEL_PATH, compile=False, custom_objects={'swish': swish, 'FixedDropout': FixedDropout})
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)

# Load and preprocess the image
im = Image.open('WhatsApp Image 2024-10-24 at 21.05.28.jpeg')

# Assuming preprocess function and input_size are defined
X = preprocess(im, input_size)  # Modify input_size as needed
X = np.reshape([X], (1, *X.shape))  # Adjust shape for prediction if necessary
y = model.predict(X)

# Assuming labels list is defined
print(labels[np.argmax(y)], np.max(y))


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 803ms/step
10k 0.9998927


In [15]:
# KONVERSI .h5 to .tflite

In [24]:
from tensorflow.keras.layers import Dropout

# Define the custom FixedDropout layer if needed
class FixedDropout(Dropout):
    def __init__(self, rate, **kwargs):
        super(FixedDropout, self).__init__(rate, **kwargs)

# Add both 'swish' and 'FixedDropout' to custom_objects
custom_objects = {
    'swish': swish,
    'FixedDropout': FixedDropout
}

# Load the model with custom_objects
model = tf.keras.models.load_model("model/medium_project/model_medium.h5", custom_objects=custom_objects)


In [26]:
import tensorflow as tf
from tensorflow.keras.activations import swish  # Import swish activation

# Muat model .h5 dengan custom_objects
model = tf.keras.models.load_model("model/medium_project/model_medium.h5", custom_objects={'swish': swish})

# Konversi ke TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model TFLite
with open("model/medium_project/model_medium.tflite", "wb") as f:
    f.write(tflite_model)


ValueError: Unknown layer: 'FixedDropout'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.